<a href="https://colab.research.google.com/github/simulate111/General/blob/main/Turku_Finland_MeteoStat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [30]:
pip install meteostat

In [32]:
from meteostat import Point, Hourly
from datetime import datetime
import pandas as pd
import matplotlib.pyplot as plt

# --- CONFIGURATION ---
# Turku Coordinates (Latitude, Longitude)
LAT = 60.4518
LON = 22.2666
START = datetime(2024, 1, 1)
END = datetime(2025, 1, 1)

print("Fetching data from Meteostat (Ground Stations)...")

# 1. Define the Location (Turku)
location = Point(LAT, LON)

# 2. Fetch Hourly Data
# Meteostat automatically finds the nearest station (likely Turku Artukainen)
data = Hourly(location, START, END)
data = data.fetch()

if not data.empty:
    # 3. Clean and Format
    df = data.reset_index()

    # Rename columns to match your preferred format
    # Meteostat columns: temp (C), dwpt (Dew Point), rhum (Humidity),
    # prcp (Precip), snow (Snow), wdir (Wind Dir), wspd (Wind Speed km/h),
    # wpgt (Gust), pres (Pressure), tsun (Sunshine min), coco (Condition)

    df = df.rename(columns={
        'time': 'Time',
        'temp': 'Temperature_C',
        'wspd': 'Wind_Speed_kmh' # Meteostat uses km/h, NOT m/s
    })

    # Conversion: km/h to m/s
    df['Wind_Speed_ms'] = df['Wind_Speed_kmh'] / 3.6

    # Formatting Dates
    df['Date'] = df['Time'].dt.strftime('%Y-%m-%d')
    df['Hour'] = df['Time'].dt.strftime('%H:%M')

    # Select Columns
    # Note: We check if 'rad' (Radiation) exists, but it's usually empty for airports
    cols = ['Date', 'Hour', 'Temperature_C', 'Wind_Speed_ms']
    if 'rad' in df.columns:
        df['GHI_Wm2'] = df['rad'] # If available
        cols.append('GHI_Wm2')

    df_final = df[cols]

    print("-" * 30)
    print(df_final.head())
    print("-" * 30)
    print(f"Total Rows: {len(df_final)}")

    filename = "Turku_Meteostat_2024.csv"
    df_final.to_csv(filename, index=False)
    print(f"Success! Saved to {filename}")

    # Check if Radiation was actually found
    if 'GHI_Wm2' not in df_final.columns or df_final['GHI_Wm2'].isna().all():
        print("\n⚠️ WARNING: The 'rad' (Radiation) column is missing or empty.")
        print("   Meteostat relies on airport reports, and many don't report solar data.")
        print("   For GHI, use the Open-Meteo script (Satellite) I gave you previously.")

else:
    print("No data found for this location/time.")

Fetching data from Meteostat (Ground Stations)...
------------------------------
         Date   Hour  Temperature_C  Wind_Speed_ms
0  2024-01-01  00:00          -15.7       3.388889
1  2024-01-01  01:00          -16.0       3.194444
2  2024-01-01  02:00          -16.4       3.388889
3  2024-01-01  03:00          -16.4       3.694444
4  2024-01-01  04:00          -16.0            4.0
------------------------------
Total Rows: 8785
Success! Saved to Turku_Meteostat_2024.csv

⚠️ WARNING: The 'rad' (Radiation) column is missing or empty.
   Meteostat relies on airport reports, and many don't report solar data.
   For GHI, use the Open-Meteo script (Satellite) I gave you previously.
